In [1]:
from utils.molecule_tools import construct_molecule
from utils.cs_vqe_tools_original import greedy_dfs
from cs_vqe.circuit import cs_vqe_circuit
import json

Will store matrices in sparse form


/home/tweaving/.local/lib/python3.8/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')
/home/tweaving/.local/lib/python3.8/site-packages/qiskit/aqua/components/optimizers/optimizer.py:49: DeprecationWarning: The package qiskit.aqua.components.optimizers is deprecated. It was moved/refactored to qiskit.algorithms.optimizers (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua.components.optimizers',


In [2]:
file = 'molecule_data'
with open('data/'+file+'.json', 'r') as json_file:
    molecule_data = json.load(json_file)

with open('data/anz_circ_depth.json', 'r') as json_file:
    anz_data = json.load(json_file)
    
speciesname_list = [mol for mol in anz_data if (anz_data[mol]['num_qubits']!=anz_data[mol]['chemaccnum'] and not anz_data[mol]['chemaccnum']==0) and mol.find('alt')==-1 and mol.find('+')==-1]
speciesname_list

['H2O_STO-3G_SINGLET',
 'LiH_STO-3G_SINGLET',
 'HF_STO-3G_SINGLET',
 'CH2_STO-3G_SINGLET',
 'BeH2_STO-3G_SINGLET',
 'Be_STO-3G_SINGLET',
 'C_STO-3G_SINGLET',
 'NH_STO-3G_SINGLET',
 'BH_STO-3G_SINGLET']

In [3]:
model_data = {}

for speciesname in speciesname_list:
    print(speciesname)
    
    atoms, coords, multiplicity, charge, basis, sym_sector = molecule_data[speciesname].values()
    molecule3   = construct_molecule(atoms, coords, charge, multiplicity, basis, ucc_threshold=1e-3, taper=True, sym_sector=sym_sector)
    molecule5   = construct_molecule(atoms, coords, charge, multiplicity, basis, ucc_threshold=1e-5, taper=True, sym_sector=sym_sector)
    num_qubits  = molecule3['num_qubits']
    ham         = molecule3['hamiltonian']
    uccsd3      = molecule3['uccsdansatz']
    uccsd5      = molecule5['uccsdansatz']
    hf_config   = molecule3['hf_config']
    num_tapered = molecule3['num_tapered']
    true_gs_nrg = molecule3['true_gs_nrg']
    #true_gs_vec = molecule['true_gs_vec']

    for p in ham.keys():
        ham[p] = ham[p].real    
    for p in uccsd3.keys():
        uccsd3[p] = uccsd3[p].imag
    for p in uccsd5.keys():
        uccsd5[p] = uccsd5[p].imag

    terms_noncon = greedy_dfs(ham, 10, criterion='size')[-1]

    print('Number of qubits:', num_qubits)
    mol_circ = cs_vqe_circuit(hamiltonian = ham,
                              terms_noncon= terms_noncon,
                              num_qubits  = num_qubits,
                              hf_config   = hf_config)

    ham_rotations  = mol_circ.ham_rotations
    noncon         = mol_circ.noncon
    truegs         = mol_circ.truegs
    G              = mol_circ.G
    A              = mol_circ.A
    X_index        = mol_circ.X_index
    X_qubit        = mol_circ.X_qubit
    cs_vqe_energy  = mol_circ.cs_vqe_energy
    cs_vqe_errors  = mol_circ.cs_vqe_errors
    chem_acc_num_q = mol_circ.chem_acc_num_q
    order          = mol_circ.order
    ham_reduced    = mol_circ.ham_reduced
    reference_state= mol_circ.reference_state()

    model_data[speciesname] = {"ham":ham,
                               "uccsd3":uccsd3,
                               "uccsd5":uccsd5,
                               "num_qubits":num_qubits,
                               "hf_config":hf_config,
                               "terms_noncon":terms_noncon,
                               "num_tapered":num_tapered,
                               "true_gs_nrg":true_gs_nrg,
                               #"true_gs_vec":true_gs_vec,
                               "ham_rotations":ham_rotations,
                               "noncon":noncon,
                               "truegs":truegs,
                               "G":G,
                               "A":A,
                               "X_index":X_index,
                               "X_qubit":X_qubit,
                               "cs_vqe_energy":cs_vqe_energy,
                               "cs_vqe_errors":cs_vqe_errors,
                               "chem_acc_num_q":chem_acc_num_q,
                               "order":order,
                               "ham_reduced":ham_reduced,
                               "reference_state":reference_state
                                }

    print('Molecule constructed.\n')


H2O_STO-3G_SINGLET
Attempting to taper 14 --> 10 qubits
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Energies match in sector (1, -1, -1, 1), tapering successful!

Attempting to taper 14 --> 10 qubits
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Energies match in sector (1, -1, -1, 1), tapering successful!

Number of qubits: 10
Molecule constructed.

LiH_STO-3G_SINGLET
Attempting to taper 12 --> 8 qubits
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Energies match in sector (1, 1, 1, 1), tapering successful!

Attempting to taper 12 --> 8 qubits
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Energies match in sector (1, 1, 1, 1), tapering successful!

Number of qubits: 8
Molecule constructed.

HF_STO-3G_SINGLET
Attempting to taper 12 --> 8 qubits
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Energies match in sector (1, 1, -1, -1), tapering successful!

Attempting to taper 12 --> 8 qubits
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Energies match in sector (1, 1, -1, -1), tapering successful!

Number of qubits: 8
Molecule constructed.



In [8]:
#data={}
#for k in model_data.keys():
#    data[str(k)] = model_data[k]
     
with open("data/model_data.json", "w") as outfile:
    json.dump(model_data, outfile)